In [1]:
import os
import pandas as pd

# Define the base directory
base_dir = "/Users/aarohiverma/Documents/Chameleon/test/"

# Initialize lists to hold the paths and labels
data = []

# Loop over the two subfolders
for label_str, label_int in [("0_real", 0), ("1_fake", 1)]:
    folder_path = os.path.join(base_dir, label_str)
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):  # add more if needed
            full_path = os.path.join(folder_path, filename)
            data.append((full_path, label_int))

# Create the DataFrame
df = pd.DataFrame(data, columns=['path', 'label'])

# Optional: shuffle the DataFrame
df = df.sample(frac=1).reset_index(drop=True)

# Display the first few rows
df.head()
df.to_csv("chameleon.csv", index=False)

# Model training on original dataset

In [2]:
import torch
import clip
import pandas as pd
import numpy as np
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14", device=device)

# Modify preprocessing (larger image size, optional)
custom_preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    preprocess.transforms[-3],
    preprocess.transforms[-2],
    preprocess.transforms[-1],
])

# Load and shuffle dataset
df = pd.read_csv("/kaggle/input/detect-ai-vs-human-generated-images/train.csv")
df = df.sample(frac=1, random_state=42).reset_index(drop=True)  # Shuffle

# Manual 80-20 split
split_idx = int(0.8 * len(df))
df_80 = df.iloc[:split_idx].reset_index(drop=True)
df_20 = df.iloc[split_idx:].reset_index(drop=True)

# Dataset class
class ImageDataset(Dataset):
    def __init__(self, df, base_path, transform):
        self.df = df
        self.base_path = base_path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = f"{self.base_path}/{self.df.iloc[idx]['file_name']}"
        label = self.df.iloc[idx]['label']
        image = self.transform(Image.open(img_path).convert("RGB"))
        return image, label

# Feature extraction function
def extract_features(df_split, name):
    dataset = ImageDataset(df_split, "/kaggle/input/ai-vs-human-generated-dataset", custom_preprocess)
    dataloader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=2)

    features = []
    labels = []

    with torch.no_grad():
        for images, lbls in dataloader:
            images = images.to(device)
            batch_features = model.encode_image(images).cpu().numpy()
            features.append(batch_features)
            labels.append(lbls.numpy())

    X = np.vstack(features)
    y = np.concatenate(labels)

    np.save(f"clip_features_{name}.npy", X)
    np.save(f"clip_labels_{name}.npy", y)
    print(f"Feature extraction completed for {name} set!")

# Run for both splits
extract_features(df_80, "80_448")
extract_features(df_20, "20_448")


100%|███████████████████████████████████████| 890M/890M [00:34<00:00, 27.1MiB/s]


Feature extraction completed for 80_448 set!
Feature extraction completed for 20_448 set!


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Load CLIP features and labels
features = np.load("/kaggle/input/clip-embeddings-test/clip_features_80_448.npy")
labels = np.load("/kaggle/input/clip-embeddings-test/clip_labels_80_448.npy")

# Normalize Features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features = scaler.fit_transform(features)

indices = np.random.permutation(len(features))
features = features[indices]
labels = labels[indices] 

# Convert to Torch Tensors
features = torch.tensor(features, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)  # Ensure it's (N,1)

# Split into Train & Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create PyTorch DataLoaders
batch_size = 64
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)

# Define MLP Model
class MLPClassifier(nn.Module):
    def __init__(self, input_dim):
        super(MLPClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.6),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.6),

            nn.Linear(256, 256),  # 🔹 Keep 256 Instead of 128
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.7),

            nn.Linear(256, 1),
            nn.Sigmoid()
        )


    def forward(self, x):
        return self.model(x)


# Initialize Model
input_dim = features.shape[1]  # Either 768 or 1024
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLPClassifier(input_dim).to(device)

# Loss & Optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy for Classification
optimizer = optim.AdamW(model.parameters(), lr=1e-4)  # 🔹 AdamW with weight decay

# Training Loop
num_epochs = 40
for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        # Metrics
        total_loss += loss.item()
        preds = (outputs > 0.5).float()
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}, Accuracy: {acc:.2f}%")

Epoch 1/40, Loss: 0.3836, Accuracy: 81.45%
Epoch 2/40, Loss: 0.1396, Accuracy: 94.89%
Epoch 3/40, Loss: 0.1003, Accuracy: 96.26%
Epoch 4/40, Loss: 0.0837, Accuracy: 96.95%
Epoch 5/40, Loss: 0.0730, Accuracy: 97.31%
Epoch 6/40, Loss: 0.0657, Accuracy: 97.59%
Epoch 7/40, Loss: 0.0608, Accuracy: 97.81%
Epoch 8/40, Loss: 0.0562, Accuracy: 97.94%
Epoch 9/40, Loss: 0.0528, Accuracy: 98.08%
Epoch 10/40, Loss: 0.0472, Accuracy: 98.31%
Epoch 11/40, Loss: 0.0432, Accuracy: 98.42%
Epoch 12/40, Loss: 0.0413, Accuracy: 98.51%
Epoch 13/40, Loss: 0.0377, Accuracy: 98.62%
Epoch 14/40, Loss: 0.0363, Accuracy: 98.67%
Epoch 15/40, Loss: 0.0322, Accuracy: 98.85%
Epoch 16/40, Loss: 0.0317, Accuracy: 98.86%
Epoch 17/40, Loss: 0.0315, Accuracy: 98.86%
Epoch 18/40, Loss: 0.0297, Accuracy: 98.95%
Epoch 19/40, Loss: 0.0278, Accuracy: 99.00%
Epoch 20/40, Loss: 0.0293, Accuracy: 98.91%
Epoch 21/40, Loss: 0.0268, Accuracy: 99.03%
Epoch 22/40, Loss: 0.0284, Accuracy: 98.99%
Epoch 23/40, Loss: 0.0254, Accuracy: 99.0

# Few shot training base model on Chameleon dataset

In [ ]:
import torch
import clip
import pandas as pd
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Load CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model, preprocess = clip.load("ViT-L/14", device=device)

# Custom preprocessing (preserving CLIP normalization)
custom_preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    preprocess.transforms[-3],  # ToTensor
    preprocess.transforms[-2],  # Normalize
    preprocess.transforms[-1],  # (Optional) Additional CLIP-specific step
])

# Replace with your actual dataframe
df = pd.read_csv("chameleon.csv")
# Ensure df has columns: 'path' and 'label'

# Shuffle and split
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
split_idx = int(0.8 * len(df))
df_80 = df.iloc[:split_idx].reset_index(drop=True)
df_20 = df.iloc[split_idx:].reset_index(drop=True)

# Dataset class
class LocalImageDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.iloc[idx]['path']
        label = self.df.iloc[idx]['label']
        image = Image.open(path).convert("RGB")
        image = self.transform(image)
        return image, label

# Feature extractor
def extract_clip_features(df_split, name):
    dataset = LocalImageDataset(df_split, custom_preprocess)
    dataloader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=0)  # ✅ Set to 0 for macOS

    features, labels = [], []

    with torch.no_grad():
        for i, (imgs, lbls) in enumerate(dataloader):
            print(f"Batch {i+1}/{len(dataloader)}")
            imgs = imgs.to(device)
            clip_feats = model.encode_image(imgs).cpu().numpy()
            features.append(clip_feats)
            labels.append(lbls.numpy())

    X = np.vstack(features)
    y = np.concatenate(labels)

    np.save(f"chameleon_clip_features_{name}.npy", X)
    np.save(f"chameleon_clip_labels_{name}.npy", y)
    print(f"✔️ Saved clip_features_{name}.npy and clip_labels_{name}.npy")

# Extract features
extract_clip_features(df_80, "train")
extract_clip_features(df_20, "test")


In [2]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

# ------------------------------
# Load NEW Training Data
# ------------------------------
features_new = np.load("/kaggle/input/chameleon-few-shot/chameleon_clip_features_train.npy")
labels_new = np.load("/kaggle/input/chameleon-few-shot/chameleon_clip_labels_train.npy")

# Normalize
scaler = StandardScaler()
features_new = scaler.fit_transform(features_new)

# Shuffle
indices = np.random.permutation(len(features_new))
features_new = features_new[indices]
labels_new = labels_new[indices]

# Convert to tensors
features_new = torch.tensor(features_new, dtype=torch.float32)
labels_new = torch.tensor(labels_new, dtype=torch.float32).unsqueeze(1)

# Create DataLoader
train_loader_new = DataLoader(TensorDataset(features_new, labels_new), batch_size=64, shuffle=True)

# Make sure model, criterion, optimizer are already defined and not reinitialized

optimizer = torch.optim.Adam(model.parameters(), lr=1e-6)  # Lower learning rate

# Make sure model, criterion, optimizer are already defined and not reinitialized
num_epochs_finetune = 100  # Or however many you want

for epoch in range(num_epochs_finetune):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for X_batch, y_batch in train_loader_new:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        preds = (outputs > 0.5).float()
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)
        total_loss += loss.item()

    acc = 100 * correct / total
    print(f"[Finetune] Epoch {epoch+1}/{num_epochs_finetune}, Loss: {total_loss/len(train_loader_new):.4f}, Accuracy: {acc:.2f}%")

[Finetune] Epoch 1/200, Loss: 5.3319, Accuracy: 70.28%
[Finetune] Epoch 2/200, Loss: 4.8904, Accuracy: 71.83%
[Finetune] Epoch 3/200, Loss: 4.3315, Accuracy: 72.99%
[Finetune] Epoch 4/200, Loss: 4.1391, Accuracy: 74.55%
[Finetune] Epoch 5/200, Loss: 3.5539, Accuracy: 75.76%
[Finetune] Epoch 6/200, Loss: 3.3672, Accuracy: 76.39%
[Finetune] Epoch 7/200, Loss: 2.9168, Accuracy: 77.51%
[Finetune] Epoch 8/200, Loss: 2.7403, Accuracy: 78.73%
[Finetune] Epoch 9/200, Loss: 2.4909, Accuracy: 79.19%
[Finetune] Epoch 10/200, Loss: 2.4405, Accuracy: 80.01%
[Finetune] Epoch 11/200, Loss: 2.2548, Accuracy: 80.68%
[Finetune] Epoch 12/200, Loss: 2.0536, Accuracy: 81.28%
[Finetune] Epoch 13/200, Loss: 1.8649, Accuracy: 81.72%
[Finetune] Epoch 14/200, Loss: 1.8843, Accuracy: 82.21%
[Finetune] Epoch 15/200, Loss: 1.6699, Accuracy: 83.04%
[Finetune] Epoch 16/200, Loss: 1.5740, Accuracy: 83.83%
[Finetune] Epoch 17/200, Loss: 1.5165, Accuracy: 83.87%
[Finetune] Epoch 18/200, Loss: 1.4103, Accuracy: 84.13%
[

# Testing on remaining Chameleon dataset

In [10]:
%%time

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load 20% features and labels
features_20 = np.load("/kaggle/input/chameleon-few-shot/chameleon_clip_features_test.npy")
labels_20 = np.load("/kaggle/input/chameleon-few-shot/chameleon_clip_labels_test.npy")

# Apply the same scaler used during training
features_20 = scaler.transform(features_20)

# Convert to torch tensors
features_20 = torch.tensor(features_20, dtype=torch.float32).to(device)
labels_20 = torch.tensor(labels_20, dtype=torch.float32).unsqueeze(1).to(device)

# Evaluate
model.eval()
with torch.no_grad():
    outputs = model(features_20)
    preds = (outputs > 0.5).float()

# Move to CPU and flatten
y_true = labels_20.cpu().numpy().flatten()
y_pred = preds.cpu().numpy().flatten()

# Metrics
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred)
rec = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Evaluation on cham 20% :")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")


Evaluation on cham 20% :
Accuracy:  0.9430
Precision: 0.9897
Recall:    0.8764
F1 Score:  0.9296
CPU times: user 187 ms, sys: 46.5 ms, total: 233 ms
Wall time: 149 ms


### Test chameleon set on other benchmark models

In [6]:
"""
chameleon_benchmark_train_eval.py
---------------------------------
Fine-tune and evaluate common deepfake detection baselines (ResNet50, EffNetV2-B3,
ViT-B/16, CLIP-ViT-L/14) on the Chameleon dataset.

Expected CSV columns: filepath,label
"""

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import CLIPModel
import timm

# ============== CONFIG ==============
CSV_PATH = "chameleon.csv"   # has filepath,label
BATCH_SIZE = 8               # reduce to prevent OOM
EPOCHS = 3                   # quick fine-tune; adjust as needed
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SPLIT_RATIO = 0.5            # 50% train / 50% test
# ====================================


# ---------- Dataset ----------
class ImageDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.df = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.iloc[idx]["filepath"]
        label = int(self.df.iloc[idx]["label"])
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label


# ---------- Transforms ----------
train_tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

test_tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])


# ---------- Utils ----------
def get_benchmark_models():
    models_dict = {}
    available = timm.list_models(pretrained=True)

    def safe_timm_model(preferred_names, fallback="efficientnet_b3"):
        """Try multiple model names safely."""
        for name in preferred_names:
            if name in available:
                print(f"✅ Using model: {name}")
                return timm.create_model(name, pretrained=True)
        print(f"⚠️ None of {preferred_names} found. Falling back to {fallback}")
        return timm.create_model(fallback, pretrained=True)

    # ResNet50
    m1 = models.resnet50(weights="IMAGENET1K_V2")
    m1.fc = nn.Linear(m1.fc.in_features, 1)
    models_dict["ResNet50"] = m1

    # EfficientNetV2-B3 (timm)
    m2 = safe_timm_model([
        "efficientnetv2_b3",
        "tf_efficientnetv2_s.in1k",
        "efficientnet_b3"
    ])
    in_features = m2.get_classifier().in_features
    m2.classifier = nn.Linear(in_features, 1)
    models_dict["EffNetV2-B3"] = m2

    # ViT-B/16
    m3 = safe_timm_model([
        "vit_base_patch16_224",
        "vit_base_patch16_224.augreg_in21k",
        "vit_base_patch16_224.in1k"
    ])
    in_features = m3.head.in_features
    m3.head = nn.Linear(in_features, 1)
    models_dict["ViT-B/16"] = m3

    # CLIP ViT-L/14
    clip = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
    visual = clip.vision_model
    head = nn.Sequential(
        nn.AdaptiveAvgPool2d((1, 1)),
        nn.Flatten(),
        nn.Linear(visual.config.hidden_size, 1)
    )
    model = nn.Sequential(visual, head)
    models_dict["CLIP-ViT-L/14"] = model

    return models_dict


def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for imgs, labels in tqdm(loader, desc="Training", leave=False):
        imgs, labels = imgs.to(DEVICE), labels.float().unsqueeze(1).to(DEVICE)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


def evaluate_model(model, loader):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Testing", leave=False):
            imgs = imgs.to(DEVICE)
            outputs = model(imgs)
            probs = torch.sigmoid(outputs).cpu().numpy().ravel()
            preds.extend((probs > 0.5).astype(int))
            trues.extend(labels.numpy())

    acc = accuracy_score(trues, preds)
    f1 = f1_score(trues, preds)
    prec = precision_score(trues, preds)
    rec = recall_score(trues, preds)
    return {"Acc": acc, "F1": f1, "Prec": prec, "Rec": rec}


# ---------- Main ----------
if __name__ == "__main__":
    # Dataset split
    full_dataset = ImageDataset(CSV_PATH, transform=train_tfm)
    n_train = int(SPLIT_RATIO * len(full_dataset))
    n_test = len(full_dataset) - n_train
    train_set, test_set = random_split(full_dataset, [n_train, n_test])

    test_set.dataset.transform = test_tfm

    # num_workers=0 avoids macOS pickle error
    train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    all_results = []
    for name, model in get_benchmark_models().items():
        print(f"\n🔹 Fine-tuning {name}")
        model = model.to(DEVICE)

        criterion = nn.BCEWithLogitsLoss()
        optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

        for epoch in range(EPOCHS):
            loss = train_one_epoch(model, train_loader, optimizer, criterion)
            print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {loss:.4f}")

        metrics = evaluate_model(model, test_loader)
        print(f"{name} → {metrics}")
        all_results.append({"Model": name, **metrics})

    df = pd.DataFrame(all_results)
    df.to_csv("benchmark_results_chameleon_finetuned.csv", index=False)
    print("\n✅ Saved fine-tuned benchmark results → benchmark_results_chameleon_finetuned.csv")
    print(df)


✅ Using model: tf_efficientnetv2_s.in1k
✅ Using model: vit_base_patch16_224.augreg_in21k

🔹 Fine-tuning ResNet50


Epoch 1/3 | Loss: 0.2649


Epoch 2/3 | Loss: 0.1384


Epoch 3/3 | Loss: 0.0924


ResNet50 → {'Acc': 0.9476837981101637, 'F1': 0.9377911756645656, 'Prec': 0.9630393996247655, 'Rec': 0.9138330069432081}

🔹 Fine-tuning EffNetV2-B3


...

Epoch 1/3 | Loss: 0.3647


Epoch 2/3 | Loss: 0.2679


Epoch 3/3 | Loss: 0.2215


ViT-B/16 → {'Acc': 0.8705538910655297, 'F1': 0.8316178674927551, 'Prec': 0.9478359908883827, 'Rec': 0.740786896920064}

🔹 Fine-tuning CLIP-ViT-L/14


AttributeError: 'BaseModelOutputWithPooling' object has no attribute 'size'